In [1]:
import sys
print(sys.version)

3.10.6 (main, Nov  2 2022, 18:53:38) [GCC 11.3.0]


In [2]:
import sklearn.ensemble
import shap
import numpy as np
import numba
import time
import xgboost
from sklearn.datasets import make_regression

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
import shap.benchmark

/home/jarek/projects/shap/venv/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
X,y = shap.datasets.boston()
X.shape

Function load_boston is deprecated; `load_boston` is deprecated in 1.0 and will be removed in 1.2.

    The Boston housing prices dataset has an ethical problem. You can refer to
    the documentation of this function for further details.

    The scikit-learn maintainers therefore strongly discourage the use of this
    dataset unless the purpose of the code is to study and educate about
    ethical issues in data science and machine learning.

    In this special case, you can fetch the dataset from the original
    source::

        import pandas as pd
        import numpy as np

        data_url = "http://lib.stat.cmu.edu/datasets/boston"
        raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
        data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
        target = raw_df.values[1::2, 2]

    Alternative datasets include the California housing dataset (i.e.
    :func:`~sklearn.datasets.fetch_california_housing`) and the Ames housing
    datase

(506, 13)

In [4]:
X, y = make_regression(n_samples=400, n_features=16, n_informative=8,random_state=0, shuffle=False)

model = sklearn.ensemble.RandomForestRegressor(n_estimators=1000, max_depth=10)
model.fit(X, y)

RandomForestRegressor(max_depth=10, n_estimators=1000)

In [5]:
# bst = xgboost.train({"learning_rate": 0.01, "max_depth": 4}, xgboost.DMatrix(X, label=y), 1000)

In [6]:
# start = time.time()
# shap_values = bst.predict(xgboost.DMatrix(X), pred_contribs=True)
# print(time.time() - start)


In [7]:
from shap.explainers.pytree import *

# x = X.head()
# x = X[:1]
x = X
x

array([[ 1.76405235,  0.40015721,  0.97873798, ...,  0.12167502,
         0.44386323,  0.33367433],
       [ 1.49407907, -0.20515826,  0.3130677 , ...,  1.46935877,
         0.15494743,  0.37816252],
       [-0.88778575, -1.98079647, -0.34791215, ..., -0.4380743 ,
        -1.25279536,  0.77749036],
       ...,
       [ 0.65717883,  0.69047822,  0.95895162, ..., -0.46446719,
        -0.93891651,  1.21233882],
       [-0.13056054,  0.21230901,  1.20128792, ..., -0.28799758,
        -1.45415099,  0.24241493],
       [ 1.22379834, -0.85122832,  1.52658725, ...,  0.34265399,
        -0.09796724, -0.24024461]])

In [8]:
from shap import TreeExplainer as TreeCext
start = time.time()
res_shap_cext = TreeCext(model).shap_values(x, banz=False)
time_shap_cext = time.time() - start
time_shap_cext

ile drzew cext? 1000
uruchamiam z BANZ=False
using treeshap and cext
here37
using dense_Tree_pred


13.3692045211792

In [9]:
start = time.time()
res_banz_cext = TreeCext(model).shap_values(x, banz=True)
time_banz_cext = time.time() - start
time_banz_cext

ile drzew cext? 1000
uruchamiam z BANZ=True
using treeshap and cext
using BANZ
using cext banz


10.391015768051147

In [10]:
model = xgboost.XGBRegressor(n_estimators=1000, subsample=0.3)
X,y = shap.datasets.boston()
X = X.values
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)
model.fit(X_train, y_train, eval_set=[(X_test, y_test)], early_stopping_rounds=10, verbose=False)

X_eval = X_test[:]
y_eval = y_test[:]

# use an independent masker
masker = shap.maskers.Independent(X_train)
pmasker = shap.maskers.Partition(X_train)


Function load_boston is deprecated; `load_boston` is deprecated in 1.0 and will be removed in 1.2.

    The Boston housing prices dataset has an ethical problem. You can refer to
    the documentation of this function for further details.

    The scikit-learn maintainers therefore strongly discourage the use of this
    dataset unless the purpose of the code is to study and educate about
    ethical issues in data science and machine learning.

    In this special case, you can fetch the dataset from the original
    source::

        import pandas as pd
        import numpy as np

        data_url = "http://lib.stat.cmu.edu/datasets/boston"
        raw_df = pd.read_csv(data_url, sep="\s+", skiprows=22, header=None)
        data = np.hstack([raw_df.values[::2, :], raw_df.values[1::2, :2]])
        target = raw_df.values[1::2, 2]

    Alternative datasets include the California housing dataset (i.e.
    :func:`~sklearn.datasets.fetch_california_housing`) and the Ames housing
    datase

In [11]:
explainers = [
    ("Tree", shap.explainers.Tree(model, masker)),
    ("TreeBanz", shap.explainers.Tree(model, masker, use_banz=True)),
    ("Tree approx.", shap.explainers.Tree(model, masker, approximate=True)),
    ("Exact", shap.explainers.Exact(model.predict, masker)),
]

ile drzew cext? 44
here37
using dense_Tree_pred
ile drzew cext? 44
here37
using dense_Tree_pred
ile drzew cext? 44
here37
using dense_Tree_pred
uruchomiono zmieniony


In [12]:
def run_explain(name, explainer, data):
    start = time.time()
    values = exp(data)
    end = time.time()
    print("name: {}, time: {}".format(name, end - start))
    return (name, values)

In [13]:
# # dry run to get all the code warmed up for valid runtime measurements
for name, exp in explainers:
    exp(X_eval[:1])

# explain with all the explainers
attributions = [run_explain(name, exp, X_eval) for name, exp in explainers]

uruchamiam z BANZ=False
using treeshap and cext
here37
using dense_Tree_pred
uruchamiam z BANZ=True
using treeshap and cext
using BANZ
uruchamiam z BANZ=False
using treeshap and cext
here37
using dense_Tree_pred
using cext banz


Exact explainer: 128it [00:15,  2.54it/s]                                                                                                                                                                                                                                                                       


name: Tree, time: 15.360111713409424


Exact explainer: 128it [00:15,  2.69it/s]                                                                                                                                                                                                                                                                       


name: TreeBanz, time: 15.96930742263794


Exact explainer: 128it [00:17,  2.84it/s]                                                                                                                                                                                                                                                                       


name: Tree approx., time: 17.635498523712158


Exact explainer: 128it [00:19,  2.78it/s]                                                                                                                                                                                                                                                                       

name: Exact, time: 19.060742616653442


In [14]:
ex = TreeExplainer(model)
start = time.time()
res_shap_py = ex.shap_values(x)
time_shap_py = time.time() - start

Exception: Model type not supported by TreeExplainer: <class 'xgboost.sklearn.XGBRegressor'>

In [ ]:
start = time.time()
res_banz_py = ex.banz_values(x)
time_banz_py = time.time() - start

In [ ]:
line = np.asarray(x[:1])
ex.brute_banz(line[0], 0)

In [ ]:
ex.brute_shap(line[0], 0)

In [ ]:
results_dict_cext = {}
results_dict_cext['banz'] = {}
results_dict_cext['shap'] = {}

In [ ]:
for i in range(1, 10):
    X, y = make_regression(n_samples=200 * i, n_features=16, n_informative=8,random_state=0, shuffle=False)

    model = sklearn.ensemble.RandomForestRegressor(n_estimators=1000 + 100 * i, max_depth=10 + i)
    model.fit(X, y)
    
    start = time.time()
    res_shap_cext = TreeCext(model).shap_values(x, banz=False)
    res_shap = time.time() - start
    
    start = time.time()
    res_banz_cext = TreeCext(model).shap_values(x, banz=True)
    res_banz = time.time() - start
    
    print(res_shap)
    print(res_banz)
    results_dict_cext['banz'][i] = res_banz
    results_dict_cext['shap'][i] = res_shap
    

In [ ]:
x = [i for i in results_dict_cext['banz']]
a1 = [results_dict_cext['banz'][i] for i in results_dict_cext['banz']]
a2 = [results_dict_cext['shap'][i] for i in results_dict_cext['shap']]

y_values = {"banz": a1, "treeshap": a2}
labels = ["BANZ", "TREESHAP"]

fig, ax = plt.subplots()
ax.stackplot(x, y_values.values(), labels=labels)

ax.legend(loc='upper left')
plt.show()

In [ ]:
results_dict_py = {}
results_dict_py['banz'] = {}
results_dict_py['shap'] = {}

In [ ]:
for i in range(1, 10):
    X, y = make_regression(n_samples=100 + 25 * i, n_features=6, n_informative=2,random_state=0, shuffle=False)

    model = sklearn.ensemble.RandomForestRegressor(n_estimators=100 + 25 * i, max_depth=4)
    model.fit(X, y)
    
    ex = TreeExplainer(model)
    print('created model')
    start = time.time()
    res_shap_py = ex.shap_values(X)
    time_shap_py = time.time() - start
    
    start = time.time()
    res_banz_py = ex.banz_values(X)
    time_banz_py = time.time() - start
    
    print(res_shap_py)
    print(res_banz_py)
    results_dict_py['banz'][i] = time_banz_py
    results_dict_py['shap'][i] = time_shap_py
    

In [ ]:
x = [i for i in results_dict_py['banz']]
a1 = [results_dict_py['banz'][i] for i in results_dict_py['banz']]
a2 = [results_dict_py['shap'][i] for i in results_dict_py['shap']]

y_values = {"banz": a1, "treeshap": a2}
labels = ["BANZ", "TREESHAP"]

fig, ax = plt.subplots()
ax.stackplot(x, y_values.values(), labels=labels)

ax.legend(loc='upper left')
plt.show()

In [ ]:
len(a1)

In [ ]:
len(a2)